In [ ]:
from unidecode import unidecode
from pyspark.sql import DataFrame
from pyspark.sql.functions import col, trim, when, upper, lower, regexp_replace, count, isnan, udf
from pyspark.sql.types import StringType, DoubleType
from pyspark.sql.functions import pandas_udf

In [ ]:

class DataQualityUtils:
    
    def __init__(self, df: DataFrame):
        self.df = df

    def remove_espacos_em_colunas(self):
        """Remove espaços no nome das colunas."""
        new_columns = [col_name.strip().replace(" ", "_") for col_name in self.df.columns]
        self.df = self.df.toDF(*new_columns)
        return self
    
    def remover_acentos(self, cols: list):
        """
        Substitui letras acentuadas por equivalentes sem acento (ex: Á -> A) nas colunas especificadas,
        usando pandas_udf para melhor performance.
        :param cols: Lista com os nomes das colunas a serem tratadas.
        """

        @pandas_udf(StringType())
        def remover_acentos_pandas(series):
            return series.apply(lambda x: unidecode(x) if x is not None else None)

        for col_name in cols:
            self.df = self.df.withColumn(col_name, remover_acentos_pandas(col(col_name)))

        return self

    def trim_strings(self):
        """Remove espaços à esquerda e à direita de todas as colunas string."""
        for c in self.df.schema.fields:
            if isinstance(c.dataType, StringType):
                self.df = self.df.withColumn(c.name, trim(col(c.name)))
        return self

    def padronizar_maiusculas(self, cols: list = []):
        """Converte colunas string para maiúsculas."""
        for col_name in cols:
            self.df = self.df.withColumn(col_name, upper(col(col_name)))
        return self

    def padronizar_minusculas(self, cols: list = []):
        """Converte colunas string para minúsculas."""
        for col_name in cols:
            self.df = self.df.withColumn(col_name, lower(col(col_name)))
        return self

    def remover_caracteres_especiais(self, cols: list = []):
        """Remove caracteres especiais de colunas string selecionadas."""
        for col_name in cols:
            self.df = self.df.withColumn(col_name, regexp_replace(col(col_name), r"[^\w\s]", ""))
        return self

    def remover_valores_nulos(self, subset: list = None):
        """Remove linhas com valores nulos nas colunas indicadas."""
        self.df = self.df.dropna(subset=subset)
        return self

    def converter_para_numerico(self, cols: list):
        """
        Converte as colunas especificadas para tipo numérico (Double).
        Valores não conversíveis serão transformados em null.
        :param cols: Lista com os nomes das colunas a serem convertidas.
        """
        for col_name in cols:
            self.df = self.df.withColumn(col_name, col(col_name).cast(DoubleType()))
        return self
    
    def preencher_nulos(self, fill_dict: dict):
        """Preenche valores nulos com valores default informados (por coluna)."""
        self.df = self.df.fillna(fill_dict)
        return self

    def remover_duplicatas(self, subset: list = None):
        """Remove registros duplicados."""
        self.df = self.df.dropDuplicates(subset=subset)
        return self

    def get_df(self):
        """Retorna o DataFrame após os tratamentos."""
        return self.df
